In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Загрузка данных из Excel файла
data = pd.read_excel('tobit.xlsx', header=None)

# Разделение данных в первом столбце на несколько столбцов
data = data[0].str.split(',', expand=True)

# Дайте новым столбцам имена в соответствии с вашими данными
data.columns = ['id', 'read', 'math', 'prog', 'apt']

# Конвертирование данных в соответствующие типы
data['read'] = pd.to_numeric(data['read'])
data['math'] = pd.to_numeric(data['math'])
data['apt'] = pd.to_numeric(data['apt'])
data['prog'] = data['prog'].str.replace('"', '')  # Удаление кавычек, если они есть

# Конвертация категориальной переменной 'prog' в dummy переменные
data = pd.get_dummies(data, columns=['prog'], drop_first=True)

# Создание бинарной переменной участия для уравнения выбора
data['participation'] = (data['apt'] > 0).astype(int)



FileNotFoundError: [Errno 2] No such file or directory: 'tobit.xlsx'

In [ ]:
# Определение переменных для уравнения выбора и уравнения исхода
selection_vars = ['read', 'math', 'prog_general', 'prog_vocational']
outcome_vars = selection_vars + ['apt']

# Уравнение выбора
X_selection = sm.add_constant(data[selection_vars])
y_selection = data['participation']

# Уравнение исхода
X_outcome = sm.add_constant(data[selection_vars])
y_outcome = data['apt']

# Пробит модель для уравнения выбора
probit_model = sm.Probit(y_selection, X_selection).fit()

# Предсказание вероятностей и вычисление инверсии миллса
data['mills_ratio'] = probit_model.predict(X_selection) * (1 - probit_model.predict(X_selection))

# OLS модель для уравнения исхода с учетом коррекции инверсии миллса
X_outcome['mills_ratio'] = data['mills_ratio']
heckit_model = sm.OLS(y_outcome, X_outcome).fit()

# Вывод результатов модели Хекмана
print(heckit_model.summary())
